<a href="https://colab.research.google.com/github/tcapelle/hackercup_rag/blob/main/rag_code_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{rag-hackercup} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

# W&B Lighting Competition - AI Hacker Cup 

</a>

[Weights & Biases](https://wandb.ai/site?utm_source=colab&utm_medium=code&utm_campaign=lightning-ai-hacker-cup) are running a 7-day Lightning Competition focussed on solving practice problems for the  [2024 NeurIPS AI Hacker Cup](https://hackercupai.github.io/) challenge.

#### Goal
The goal is to try and solve all 5 of the 2023 practice questions for the AI Hacker Cup using MistralAI's models. We’re offering free MistralAI api access via the code in this colab to get people started.

#### Competition GitHub
The competition [repo here](https://github.com/tcapelle/hackercup_rag) contains this colab, the code for the Code Generation Agent and the details on how to make a submission and the competition rules.

#### Discord
You can join the official NeurIPS AI Hacker Cup [discord here](discord.gg/wWeN9hTH32) to share ideas and discuss winning solutions.

## Prizes

Weights & Biases are giving away a pair of Meta Ray-Ban Smart Glasses for:
-  the first individual to submit code that solves 4 out of 5 correct solutions
-  the first individual to submit code that solves 5 out of 5 correct solutions

## Entry Submissions, Rules & Deadline

See the [competition README](https://github.com/tcapelle/hackercup_rag) for how to make a submissions the the competition rules.

## W&B Weave

[W&B Weave](https://weave-docs.wandb.ai/tutorial-eval?utm_source=colab&utm_medium=code&utm_campaign=lightning-ai-hacker-cup) is used in this competition to run the evaluations. It is a lightweight toolkit for tracking and evaluating LLM applications, built by Weights & Biases. 

<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/static/img/evals-hero.png" width="800" height="450">

If you want to learn more about Weave, you can [get started](https://weave-docs.wandb.ai/quickstart?utm_source=colab&utm_medium=code&utm_campaign=lightning-ai-hacker-cup) by decorating Python functions with `@weave.op`.

# Using RAG for a Code Generation Agent

This colab demonstrates how to retrieve over a dataset of coding question-answer pairs (the [CodeContests](https://huggingface.co/datasets/deepmind/code_contests) dataset from DeepMind) in order to find simlar questions that might help our Agent generate the correct solution.

A more detailed walkthough of the approach we will use in this notebook can be found in the following **[Youtube video](https://www.youtube.com/watch?v=cObBj2UpWK8)**:

<a target="_blank" href="https://www.youtube.com/watch?v=cObBj2UpWK8">
<img src="https://img.youtube.com/vi/cObBj2UpWK8/0.jpg" width="400" height="300">
</a>

## Setup 

**Note: You need to run this cell only once**
We will clone the starter-kits repo
Set the rag folder as our working directory
and install the dependencies for the project.

**You can comment out the cell after you have run it once.**

In [ ]:
# Clone the starter-kits repo
!git clone https://github.com/tcapelle/hackercup_rag
# Change directory to the rag folder. Running the next line twice in the same session will raise an error.
%cd hackercup_rag
# Install dependencies
!pip install -r requirements.txt -qq

To run this colab, create a [free Weights & Biases (W&B) account here](https://wandb.ai/site?utm_source=colab&utm_medium=code&utm_campaign=lightning-ai-hacker-cup) and then copy your API key from https://wandb.ai/authorize into the input box below when requested.

In [ ]:
import os
import weave

WEAVE_PROJECT = "ai-hacker-cup"
weave_client = weave.init(WEAVE_PROJECT)

In [ ]:
# Set the URL for the Mistral model api we'll be using
os.environ["BASE_URL"] = "http://195.242.24.252:8000/v1"

# Select MistralAI models used depending if you want a fast or strong LLM
FAST_LLM = "mistral/open-mistral-nemo-2407"
STRONG_LLM = "mistral/mistral-large-latest"
os.environ["FAST_LLM"] = FAST_LLM
os.environ["STRONG_LLM"] = STRONG_LLM

# Set the max tokens for the models and how many parallel requests to make in Weave Evaluations
os.environ["MAX_TOKENS"] = "4096"
os.environ["WEAVE_PARALLELISM"] = "1"

## Challenges Dataset
We will use the **practice** dataset from the **2023** [HackerCup dataset](https://huggingface.co/datasets/hackercupai/hackercup).

We have already processed the dataset and saved it as a [`weave.Dataset`](https://weave-docs.wandb.ai/guides/core-types/datasets/?utm_source=colab&utm_medium=code&utm_campaign=lightning-ai-hacker-cup). You can either use the Dataset by running the next cell or download the dataset using the instructions below.

We will use this challenge dataset to load some practice problems and solutions from the HackerCup dataset and evaluate our agents on it.

In [ ]:
from utils import Problem

practice_dataset_uri = "weave:///parambharat/hackercup/object/practice_dataset:R35fXf9N3FE2IOesg7bRPaPAxiE9YbpirhXO9HcHs8w"
problems_dataset = weave.ref(practice_dataset_uri).get().rows[:]
problems = list(map(lambda x: Problem(**x), problems_dataset))
problem = problems[0]  # Select the first problem

print("Sample Problem:\n\n", problem.model_dump_json(indent=2))

#### [Alternative] Download the raw challenges dataset

Alternatively, you can download the dataset by running the download script from the [submit-first-solution](https://github.com/HackerCupAI/starter-kits/tree/main/submit_first_solution). Specifically, you can run the following command to download the dataset:

```bash
python download.py --year 2023 --dataset_folder data
```


This should create a `dataset` folder with the problems and solutions. Here's an example of what the data looks like for the `dim_sum_delivery` problem from the `2023` season:

```
data/dataset/2023/practice
...
├── dim_sum_delivery.cpp
├── dim_sum_delivery.in
├── dim_sum_delivery.md
├── dim_sum_delivery.out
├── dim_sum_delivery_sample_input.txt
├── dim_sum_delivery_sample_output.txt
├── dim_sum_delivery_sol.md
...
```

Each problem has a `in`, `out`, `md`, `cpp`, and `sol` file.

The `in` file contains the input data for the problem.
The `out` file contains the expected output for the problem.
The `md` file contains the problem statement.
The `cpp` file contains the source code to the solution.
The `sol` file contains the detailed solution to the problem.
The `sample_input.txt` and `sample_output.txt` files contain the sample input and output for the problem. These are the test cases that will be available to the agent during development and evaluation.

#### Turn on logging and asyncio for notebooks

In [ ]:
import asyncio
import logging
from nest_asyncio import apply

apply()
logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

## Running a RAG + Reflection Agent

### RAG Agent with Reflection

We will combine a RAG Agent with Reflection in order to:

- Retrieve similar types of questions from the CodeContests dataset, generate a solution, reflect on the solution and test results and improve it.
- We then use this improved solution to generate new few-shot examples and repeat the process in a loop until we converge to a solution or the iteration limit is reached.

`agent.py` contains the prompts used for analysis (`ANALYSIS_INSTRUCTIONS`), reflection (`REFLECTION_INSTRUCTIONS`) and problem solving (`SOLVER_INSTRUCTIONS`) feel free to edit them to improve the system.

In [ ]:
from agent import REFLECTION_INSTRUCTIONS

print(REFLECTION_INSTRUCTIONS)

### Retriever

The code used the retrieval over the CodeContests dataset can be found in `retriever.py`. Here we'll initialise our retriever.

In [ ]:
from retriever import Retriever

retriever = Retriever()

### RAG Solver Pipeline

Here we run the code generation pipeline which:
- given a problem, retrieves similar problems from the CodeCompletions dataset
- generates candidate code for problem
- executes the code
- checks if the executed code generates the correct solution
- if the solution is correct, it terminates otherwise it retries for `max_iterations`

Note `code_execution_timeout`is used to limit the time available for the generated python code to execute as sometimes the code generated be recursive code that never terminates.

In [ ]:
from agent import rag_solver
from agent import rework_solution

@weave.op
async def rag_solver_with_reflection(
        retriever: Retriever,
        problem: Problem,
        model: str = FAST_LLM,
        temperature: float = 0.7,
        max_iterations: int = 2,
        code_execution_timeout: int = 10,
):
    num_iterations = 0
    test_report = "failed"
    solution = None
    while not test_report == "passed" and num_iterations < max_iterations:
        rag_result = await rag_solver(
            retriever=retriever,
            problem=problem,
            timeout=code_execution_timeout,
            model=model,
            temperature=temperature,
        )
        solution = rag_result["solution"]
        test_report = rag_result["test_report"]
        if test_report == "passed":
            logger.info(f"Passing solution generated successfully for problem: {problem.problem_name}")
            return rag_result
        
        logger.info(f"Solution failed, reworking solution. Problem: {problem.problem_name}")
        rework_result = await rework_solution(
            problem=problem,
            incorrect_solution=solution,
            test_report=test_report,
            model=model,
            temperature=temperature,
            timeout=code_execution_timeout,
        )
        solution = rework_result["solution"]
        test_report = rework_result["test_report"]
        if test_report == "passed":
            logger.info(f"Re-worked solution passed for problem: {problem.problem_name}")
            return {
                "solution": solution,
                "stage": "reflection",
                "test_report": test_report,
            }
        num_iterations += 1
        logger.info(f"Re-worked solution failed, trying iteration {num_iterations}. Problem: {problem.problem_name}")
    logger.info("Failed to generate a solution after {num_iterations} iterations. Problem: {problem.problem_name}")
    return {"solution": solution, "stage": "failed", "test_report": test_report}

Lets run the pipeline on 1 problem:

In [ ]:
reflection_result = await rag_solver_with_reflection(
    retriever, problem, max_iterations=2, code_execution_timeout=30
)

print("*" * 80)
print(reflection_result["solution"].source_code)
print("*" * 80)
print(reflection_result["test_report"])

# Evaluation

Now we are ready to evaluate against the expected solutions.

### Create a Weave Model
First we create a Weave ["Model"](https://weave-docs.wandb.ai/guides/core-types/models?utm_source=colab&utm_medium=code&utm_campaign=lightning-ai-hacker-cup), which has a `predict` function that Weave Evaluations will call to generate a solution. It also has various attributes that we can set to adjust the behaviour of our pipeline.

In [ ]:
class RAGReflectionAgent(weave.Model):
    retriever: Retriever
    max_iterations: int = 2
    code_execution_timeout: int = 30
    model: str = STRONG_LLM
    temperature: float = 0.7

    @weave.op
    async def predict(self, problem: Problem):
        return await rag_solver_with_reflection(
            self.retriever,
            Problem(**problem),
            model=self.model,
            temperature=self.temperature,
            max_iterations=self.max_iterations,
            code_execution_timeout=self.code_execution_timeout,
        )

### Create a Scorer and define the Evals Dataset

We expect the output of the "test_report" from our solver above to be `"passed"` if the solution is correct. You can think of `expected_result` in the `evals_dataset` as the label that the `test_report` from our solver needs to return in order to ensure the generated solution is correct.

Weave Evaluations expect a list of dictionaries. 

In [ ]:
evals_dataset = [{"problem": problem, "expected_result": "passed"} for problem in problems]

Weave Evaluations use a scorer function that returns a metric and its result in a dict. Here we define a metric that checks if the code generated by agent passed the test case

In [ ]:
@weave.op
def scorer(expected_result: str, model_output: dict) -> dict:
    if model_output is None or model_output["test_report"] is None:
        return {"solution_passed": False}
    return {"solution_passed": expected_result == model_output["test_report"]}


# Weave Evaluations take a dataset and scoring functions.
# This is a evaluation that checks if the code generated by agent passed the test
evaluator = weave.Evaluation(dataset=evals_dataset, scorers=[scorer])

### Run the Evaluation
Now we instantiate the Agent and run the evaluation. Results from the evaluation will be printed in the W&B Weave UI.

In [ ]:
# Evaluate the RAG reflection agent
tasks = []

LLM = STRONG_LLM
eval_temperature = 0.7

# Instantiate the agent, which is a subclass of `weave.Model`
rag_reflection_agent = RAGReflectionAgent(
    retriever=retriever, model=LLM, temperature=eval_temperature, code_execution_timeout=30
)

# Evaluate the agent by passing it to the evaluator
# Weave Evaluations are async, so we use `asyncio.gather` to run them in parallel
rag_reflection_results = evaluator.evaluate(rag_reflection_agent)
tasks.append(rag_reflection_results)

rag_reflection_results = await asyncio.gather(*tasks)

logger.info(rag_reflection_results)